In [1]:
import pickle
import torch
import pickle
from tqdm import tqdm
from model import Net
from helper.utils import get_ec_id_dict, esm_embedding, model_embedding_test
from helper.distance_map import get_dist_map, get_dist_map_test, get_cluster_center
import pandas as pd

dtype = torch.float32
device = torch.device("cuda")


#### Eval distance map size=(N_test_id x N_train_EC) refactor

In [2]:
args_model_name = "uniref10_train_split_1_3200"
args_test_data = "uniref10_test_split_1_curate"
args_train_data = "uniref10_train_split_1"

args_hidden_dim = 512
args_out_dim = 128
id_ec_train, ec_id_dict_train = get_ec_id_dict(
    './data/' + args_train_data + '.csv')
id_ec_test, ec_id_dict_test = get_ec_id_dict(
    './data/' + args_test_data + '.csv')

model = Net(args_hidden_dim, args_out_dim, device, dtype)
checkpoint = torch.load('./model/uniref10_train_split_1_3200.pth')
model.load_state_dict(checkpoint)

emb_train = model(esm_embedding(ec_id_dict_train, device, dtype))
emb_test = model_embedding_test(id_ec_test, model, device, dtype)
eval_dist = get_dist_map_test(
    emb_train, emb_test, ec_id_dict_train, id_ec_test, device, dtype)
eval_df = pd.DataFrame.from_dict(eval_dist)
eval_df.to_csv('./eval/distmap_' + args_test_data + '.csv')

The embedding sizes for train and test: torch.Size([25584, 128]) torch.Size([5492, 128])


100%|██████████| 4223/4223 [00:00<00:00, 16399.91it/s]


Calculating eval distance map, between 5492 test ids and 4223 train EC cluster centers


5492it [00:04, 1338.37it/s]


#### Training symmetric distance map refactor

In [4]:
args_model_name = "uniref10_train_split_0"
args_hidden_dim = 512
args_out_dim  = 128
id_ec, ec_id_dict = get_ec_id_dict('./data/' + args_model_name + '.csv')
ec_id = {key: list(ec_id_dict[key]) for key in ec_id_dict.keys()}
dist_map = pickle.load( open('./data/distance_map/' + args_model_name + '.pkl', 'rb'))
model = Net(args_hidden_dim, args_out_dim, device, dtype)
checkpoint = torch.load('./model/uniref10_train_split_0_50.pth')
model.load_state_dict(checkpoint)
esm_emb = esm_embedding(ec_id_dict, device, dtype)

<All keys matched successfully>

In [ ]:
args_train_file = "uniref10_train_split_0"
_, ec_id_dict = get_ec_id_dict('./data/' + args_train_file + '.csv')
# use cpu and high precision by default
device, dtype = torch.device("cpu"), torch.float64
esm_emb = esm_embedding(ec_id_dict, device, dtype)
esm_dist = get_dist_map(ec_id_dict, esm_emb, device, dtype)
esm_df = pd.DataFrame.from_dict(esm_dist)
# pickle.dump(esm_dist, open('./data/distance_map/' +
#             args_train_file + '.pkl', 'wb'))
esm_df.to_csv('./data/distance_map/' + args_train_file + '_df.csv')